```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/signal_extraction.ipynb>`, or a {download}`python script <converted/signal_extraction.py>` with code cells.
```

# Tutorial 2: Signal Extraction

This tutorial will cover isolation of fluorescence wave dynamics from optical mapping data. It continues from the [previous tutorial](basics.ipynb), where we have already loaded the data, viewed, and performed the motion correction.

In [ ]:
import optimap as om
import numpy as np

We first repeat the steps from the previous tutorial to load the experimental data and perform motion correction.

In [ ]:
filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
video = om.load_video(filepath)
warped = om.motion_compensate(video,
                              contrast_kernel=5,
                              presmooth_spatial=1,
                              presmooth_temporal=1)
om.video.play2(video,
               warped,
               title1="with motion",
               title2="without motion",
               skip_frame=3);

In [ ]:
# Hidden, same as above but shorten the video slightly
filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
video = om.load_video(filepath)
video = video[:1000]
warped = om.motion_compensate(video,
                              contrast_kernel=5,
                              presmooth_spatial=1,
                              presmooth_temporal=1)
render_ani_func(lambda: om.video.play2(video, warped, title1="with motion", title2="without motion", skip_frame=1))

## Fluorescence wave isolation

```{warning}
This tutorial is currently work in progress. We will add more information soon.
```

We will cover two possible ways to extract the fluorescence wave dynamics:

1. Using sliding window normalization. This method conserves the shape of the action potential and normalizes the signal to [0, 1] using a sliding window. However, it can be sensitive to noise and changes of the base fluorescence due to inhomogeneous illumination when using motion correction.
2. Using a frame difference approach. This method is more robust, however it does not conserve the shape of the action potential. It is useful to visualize the propagation the front of the action potential waves.


### Sliding window normalization

The sliding/rolling window normalization performs a pixel-based normalization of the signal to [0, 1] within a short temporal window for each time point. The function {func}`video.normalize_pixelwise_slidingwindow` performs this operation, by computing the following equation for each pixel $(x, y)$ and frame $t$:

$$
\begin{align}
    \text{signal}_{\text{norm}}(t, x, y) = \frac{\text{signal}(t, x, y) - \text{min}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y)}{\text{max}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y) - \text{min}_{t' \in [t - w/2, t + w/2]} \text{signal}(t', x, y)}
\end{align}
$$

where $w$ is the window size.

Let's use a window size $w$ of 60 frames (120 ms) for this example and mask out the background.

In [ ]:
mask = om.background_mask(warped[0])
norm_warped = om.video.normalize_pixelwise_slidingwindow(warped, window_size=60)
norm_warped[:, mask] = 1.0

om.video.play(norm_warped, title="sliding window normalization", interval=20);

In [ ]:
mask = om.background_mask(warped[0])
norm_warped = om.video.normalize_pixelwise_slidingwindow(warped, window_size=60)
norm_warped[:, mask] = 1.0

render_ani_func(lambda: om.video.play(norm_warped, title="sliding window normalization", interval=30))

In [ ]:
alpha = np.clip(-(2*norm_warped - 1), 0, 1)
om.video.play_with_overlay(warped, 1-norm_warped, alpha=alpha);

In [ ]:
alpha = np.clip(-(2*norm_warped - 1), 0, 1)
render_ani_func(lambda: om.video.play_with_overlay(warped[:500], (1-norm_warped)[:500], alpha=alpha[:500], interval=20))

### Temporal difference

The temporal difference approach computes the difference between frames at time $t$ and $t - n$:

$$
\begin{align}
    \text{signal}_{\text{diff}}(t, x, y) = \text{signal}(t, x, y) - \text{signal}(t - n, x, y)
\end{align}
$$

where $\Delta t$ is the time difference between frames. This method is more robust, however it does generally not conserve the shape of the action potential. Rather, it isolates the front of the fluorescent waves.

Let's use $n = 10$ for this example.

In [ ]:
diff = om.video.temporal_difference(warped, 10)
diff[diff > 0] = 0
diff[:, mask] = 0
diff = om.video.normalize_pixelwise_slidingwindow(-diff, window_size=60)
om.video.play_with_overlay(warped, diff, vmin_overlay=-1, vmax_overlay=1)

In [ ]:
diff = om.video.temporal_difference(warped[:500], 10)
diff[diff > 0] = 0
diff[:, mask] = 0
diff = om.video.normalize_pixelwise_slidingwindow(-diff, window_size=60)
render_ani_func(lambda: om.video.play_with_overlay(warped[:500], diff, vmin_overlay=-1, vmax_overlay=1, interval=20))
del diff